Question 1: Please help me find out the license thing

### first approach 

In [ ]:
%time
# PDF to HTML
# http://macappstore.org/pdf2htmlex/
# https://github.com/coolwanglu/pdf2htmlEX/wiki/Command-Line-Options
import os
echo = "pdf2htmlEX --zoom 1.3"
for filename in os.listdir("pdf_files"):
    if filename.endswith("pdf"):
        print(filename)
        #os.system(echo+" "+filename)

## second approach (current one)

In [ ]:
%time
# PDF to HTML
# https://sourceforge.net/projects/pdftohtml/
# https://www.experts-exchange.com/videos/59892/Xpdf-PDFtoHTML-Command-Line-Utility-to-Convert-a-PDF-File-to-HTML.html
# https://www.xpdfreader.com/download.html

# make sure to have a folder named "pdf_files" with the pdfs
# make sure to have an empty folder named "html_files"
import os
echo = "pdftohtml"
for filename in os.listdir("pdf_files"):
    if filename.endswith("pdf"):
        print(filename)
        file_pdf = "pdf_files/" + filename
        file_html = "html_files/" + filename
        #os.system(echo+" "+file_pdf+" "+file_html) 

## creates table of features

In [3]:
import codecs
from bs4 import BeautifulSoup
import cssutils
import pandas as pd
import matplotlib.pyplot as plt
import os

In [4]:
def style_dict(style_tem):
    style_tem = style_tem[:-1]
    style_dic = dict(item.split(":") for item in style_tem.split(";"))
    return style_dic

In [5]:
def css_style(soup):
    selectors = {}
    for styles in soup.select('style'):
        css = cssutils.parseString(styles.encode_contents())
        for rule in css:
            if rule.type == rule.STYLE_RULE:
                style = rule.selectorText
                selectors[style] = {}
                for item in rule.style:
                    propertyname = item.name
                    value = item.value
                    selectors[style][propertyname] = value

    dict_css = {key.strip('#'): item for key, item in selectors.items()}
    return dict_css

In [6]:
def extract_features(path):
    with open(path) as raw_resuls:
        results = BeautifulSoup(raw_resuls, 'lxml')
    
    dict_css = css_style(results)
    lines = []
    text = []
    for element in results.find_all("body"):
        for stat in element.find_all("div"):
            data={}
            # gets the info from <div>
            div_dic = stat.attrs
            data.update(div_dic)

            # gets info from <span>
            span_dic = stat.find('span').attrs
            data.update(span_dic)

            # gets the features from style in <div>
            style_ = span_dic.get('style')
            style_span = style_dict(style_)
            data.update(style_span)

            # gets features of the font from css 
            font = span_dic.get('id')
            temp = dict_css.get(font)
            data.update(temp)

            # gets the features from style in <div>
            style_ = div_dic.get('style')
            style_temp = style_dict(style_)
            data.update(style_temp)
            
            d = pd.DataFrame.from_dict(data)
            
            text.append(stat.text)
            #print(stat.text)
            lines.append(d)

    dataframe = pd.concat(lines, axis=0).reset_index(drop=1)
    dataframe["text"] = pd.Series( (v for v in text))
    
    return dataframe

### reads all html files

In [7]:
%time
features = []
for root, dirs, files in os.walk("html_files"):
    for file in files:
        if file.endswith(".html"):
            if file != "index.html":
                try:
                    file_path =  os.path.join(root, file)
                    print(file_path+ " found")
                    df = extract_features(file_path)
                    df = df.assign(page=file)
                    df = df.assign(document=root)
                    features.append(df)
                except:
                    print(file_path+" NOT found")

Wall time: 0 ns
html_files\CMS_2014_0115_0059.pdf\page1.html found
html_files\CMS_2014_0115_0059.pdf\page1.html NOT found
html_files\CMS_2014_0115_0059.pdf\page2.html found
html_files\CMS_2014_0115_0059.pdf\page2.html NOT found
html_files\CMS_2014_0115_0059.pdf\page3.html found
html_files\CMS_2014_0115_0059.pdf\page3.html NOT found
html_files\CMS_2014_0115_0059.pdf\page4.html found
html_files\CMS_2014_0115_0059.pdf\page4.html NOT found
html_files\CMS_2014_0115_0059.pdf\page5.html found
html_files\CMS_2014_0115_0059.pdf\page5.html NOT found
html_files\CMS_2014_0115_0059.pdf\page6.html found
html_files\CMS_2014_0115_0076.pdf\page1.html found
html_files\CMS_2014_0115_0076.pdf\page1.html NOT found
html_files\CMS_2014_0115_0076.pdf\page2.html found
html_files\CMS_2014_0115_0076.pdf\page2.html NOT found
html_files\CMS_2014_0115_0076.pdf\page3.html found
html_files\CMS_2014_0115_0076.pdf\page3.html NOT found
html_files\CMS_2014_0115_0076.pdf\page4.html found
html_files\CMS_2014_0115_0076.pdf\

html_files\CMS_2014_0115_12912.pdf\page6.html found
html_files\CMS_2014_0115_12912.pdf\page6.html NOT found
html_files\CMS_2014_0115_12912.pdf\page7.html found
html_files\CMS_2014_0115_12938.pdf\page1.html found
html_files\CMS_2014_0115_12938.pdf\page1.html NOT found
html_files\CMS_2014_0115_12938.pdf\page10.html found
html_files\CMS_2014_0115_12938.pdf\page10.html NOT found
html_files\CMS_2014_0115_12938.pdf\page11.html found
html_files\CMS_2014_0115_12938.pdf\page11.html NOT found
html_files\CMS_2014_0115_12938.pdf\page2.html found
html_files\CMS_2014_0115_12938.pdf\page3.html found
html_files\CMS_2014_0115_12938.pdf\page3.html NOT found
html_files\CMS_2014_0115_12938.pdf\page4.html found
html_files\CMS_2014_0115_12938.pdf\page4.html NOT found
html_files\CMS_2014_0115_12938.pdf\page5.html found
html_files\CMS_2014_0115_12938.pdf\page5.html NOT found
html_files\CMS_2014_0115_12938.pdf\page6.html found
html_files\CMS_2014_0115_12938.pdf\page6.html NOT found
html_files\CMS_2014_0115_129

html_files\CMS_2014_0115_13244.pdf\page1.html found
html_files\CMS_2014_0115_13244.pdf\page1.html NOT found
html_files\CMS_2014_0115_13244.pdf\page2.html found
html_files\CMS_2014_0115_13244.pdf\page2.html NOT found
html_files\CMS_2014_0115_13244.pdf\page3.html found
html_files\CMS_2014_0115_13244.pdf\page3.html NOT found
html_files\CMS_2014_0115_13244.pdf\page4.html found
html_files\CMS_2014_0115_13244.pdf\page4.html NOT found
html_files\CMS_2014_0115_13244.pdf\page5.html found
html_files\CMS_2014_0115_13244.pdf\page6.html found
html_files\CMS_2014_0115_13244.pdf\page6.html NOT found
html_files\CMS_2014_0115_13244.pdf\page7.html found
html_files\CMS_2014_0115_13244.pdf\page7.html NOT found
html_files\CMS_2014_0115_13245.pdf\page1.html found
html_files\CMS_2014_0115_13245.pdf\page1.html NOT found
html_files\CMS_2014_0115_13245.pdf\page2.html found
html_files\CMS_2014_0115_13245.pdf\page3.html found
html_files\CMS_2014_0115_13245.pdf\page3.html NOT found
html_files\CMS_2014_0115_13245.p

html_files\CMS_2014_0115_51400.pdf\page3.html NOT found
html_files\CMS_2014_0115_51400.pdf\page4.html found
html_files\CMS_2014_0115_51400.pdf\page4.html NOT found
html_files\CMS_2014_0115_51400.pdf\page5.html found
html_files\CMS_2014_0115_51400.pdf\page5.html NOT found
html_files\CMS_2014_0115_51400.pdf\page6.html found
html_files\CMS_2014_0115_51400.pdf\page6.html NOT found
html_files\CMS_2014_0115_51400.pdf\page7.html found
html_files\CMS_2014_0115_51400.pdf\page7.html NOT found
html_files\CMS_2014_0115_51400.pdf\page8.html found
html_files\CMS_2014_0115_51400.pdf\page8.html NOT found
html_files\CMS_2014_0115_51400.pdf\page9.html found
html_files\CMS_2014_0115_51400.pdf\page9.html NOT found
html_files\CMS_2014_0115_51425.pdf\page1.html found
html_files\CMS_2014_0115_51425.pdf\page1.html NOT found
html_files\CMS_2014_0115_51425.pdf\page2.html found
html_files\CMS_2014_0115_51425.pdf\page2.html NOT found
html_files\CMS_2014_0115_51425.pdf\page3.html found
html_files\CMS_2014_0115_514

html_files\CMS_2014_0115_52959.pdf\page12.html found
html_files\CMS_2014_0115_52959.pdf\page13.html found
html_files\CMS_2014_0115_52959.pdf\page14.html found
html_files\CMS_2014_0115_52959.pdf\page15.html found
html_files\CMS_2014_0115_52959.pdf\page16.html found
html_files\CMS_2014_0115_52959.pdf\page17.html found
html_files\CMS_2014_0115_52959.pdf\page18.html found
html_files\CMS_2014_0115_52959.pdf\page19.html found
html_files\CMS_2014_0115_52959.pdf\page2.html found
html_files\CMS_2014_0115_52959.pdf\page20.html found
html_files\CMS_2014_0115_52959.pdf\page21.html found
html_files\CMS_2014_0115_52959.pdf\page22.html found
html_files\CMS_2014_0115_52959.pdf\page23.html found
html_files\CMS_2014_0115_52959.pdf\page24.html found
html_files\CMS_2014_0115_52959.pdf\page25.html found
html_files\CMS_2014_0115_52959.pdf\page3.html found
html_files\CMS_2014_0115_52959.pdf\page4.html found
html_files\CMS_2014_0115_52959.pdf\page5.html found
html_files\CMS_2014_0115_52959.pdf\page6.html foun

...

html_files\CMS_2014_0115_56370.pdf\page3.html found
html_files\CMS_2014_0115_56370.pdf\page4.html found
html_files\CMS_2014_0115_56370.pdf\page5.html found
html_files\CMS_2014_0115_56370.pdf\page6.html found
html_files\CMS_2014_0115_56380.pdf\page1.html found
html_files\CMS_2014_0115_56380.pdf\page1.html NOT found
html_files\CMS_2014_0115_56380.pdf\page2.html found
html_files\CMS_2014_0115_56380.pdf\page2.html NOT found
html_files\CMS_2014_0115_56380.pdf\page3.html found
html_files\CMS_2014_0115_56380.pdf\page3.html NOT found
html_files\CMS_2014_0115_56380.pdf\page4.html found
html_files\CMS_2014_0115_56380.pdf\page4.html NOT found
html_files\CMS_2014_0115_56411.pdf\page1.html found
html_files\CMS_2014_0115_56411.pdf\page1.html NOT found
html_files\CMS_2014_0115_56411.pdf\page2.html found
html_files\CMS_2014_0115_56411.pdf\page2.html NOT found
html_files\CMS_2014_0115_56411.pdf\page3.html found
html_files\CMS_2014_0115_56411.pdf\page3.html NOT found
html_files\CMS_2014_0115_56411.pdf\p

html_files\CMS_2014_0115_58004.pdf\page1.html found
html_files\CMS_2014_0115_58004.pdf\page10.html found
html_files\CMS_2014_0115_58004.pdf\page10.html NOT found
html_files\CMS_2014_0115_58004.pdf\page11.html found
html_files\CMS_2014_0115_58004.pdf\page11.html NOT found
html_files\CMS_2014_0115_58004.pdf\page12.html found
html_files\CMS_2014_0115_58004.pdf\page12.html NOT found
html_files\CMS_2014_0115_58004.pdf\page13.html found
html_files\CMS_2014_0115_58004.pdf\page13.html NOT found
html_files\CMS_2014_0115_58004.pdf\page14.html found
html_files\CMS_2014_0115_58004.pdf\page2.html found
html_files\CMS_2014_0115_58004.pdf\page3.html found
html_files\CMS_2014_0115_58004.pdf\page3.html NOT found
html_files\CMS_2014_0115_58004.pdf\page4.html found
html_files\CMS_2014_0115_58004.pdf\page4.html NOT found
html_files\CMS_2014_0115_58004.pdf\page5.html found
html_files\CMS_2014_0115_58004.pdf\page5.html NOT found
html_files\CMS_2014_0115_58004.pdf\page6.html found
html_files\CMS_2014_0115_58

html_files\CMS_2014_0115_58128.pdf\page4.html found
html_files\CMS_2014_0115_58128.pdf\page5.html found
html_files\CMS_2014_0115_58128.pdf\page6.html found
html_files\CMS_2014_0115_58128.pdf\page7.html found
html_files\CMS_2014_0115_58128.pdf\page8.html found
html_files\CMS_2014_0115_58128.pdf\page9.html found
html_files\CMS_2014_0115_58152.pdf\page1.html found
html_files\CMS_2014_0115_58152.pdf\page1.html NOT found
html_files\CMS_2014_0115_58152.pdf\page10.html found
html_files\CMS_2014_0115_58152.pdf\page10.html NOT found
html_files\CMS_2014_0115_58152.pdf\page11.html found
html_files\CMS_2014_0115_58152.pdf\page11.html NOT found
html_files\CMS_2014_0115_58152.pdf\page12.html found
html_files\CMS_2014_0115_58152.pdf\page12.html NOT found
html_files\CMS_2014_0115_58152.pdf\page13.html found
html_files\CMS_2014_0115_58152.pdf\page13.html NOT found
html_files\CMS_2014_0115_58152.pdf\page14.html found
html_files\CMS_2014_0115_58152.pdf\page14.html NOT found
html_files\CMS_2014_0115_58152

html_files\CMS_2014_0115_58259.pdf\page9.html found
html_files\CMS_2014_0115_58260.pdf\page1.html found
html_files\CMS_2014_0115_58260.pdf\page1.html NOT found
html_files\CMS_2014_0115_58260.pdf\page10.html found
html_files\CMS_2014_0115_58260.pdf\page10.html NOT found
html_files\CMS_2014_0115_58260.pdf\page11.html found
html_files\CMS_2014_0115_58260.pdf\page12.html found
html_files\CMS_2014_0115_58260.pdf\page12.html NOT found
html_files\CMS_2014_0115_58260.pdf\page13.html found
html_files\CMS_2014_0115_58260.pdf\page13.html NOT found
html_files\CMS_2014_0115_58260.pdf\page14.html found
html_files\CMS_2014_0115_58260.pdf\page14.html NOT found
html_files\CMS_2014_0115_58260.pdf\page15.html found
html_files\CMS_2014_0115_58260.pdf\page15.html NOT found
html_files\CMS_2014_0115_58260.pdf\page16.html found
html_files\CMS_2014_0115_58260.pdf\page16.html NOT found
html_files\CMS_2014_0115_58260.pdf\page17.html found
html_files\CMS_2014_0115_58260.pdf\page17.html NOT found
html_files\CMS_20

html_files\CMS_2014_0115_58434.pdf\page15.html found
html_files\CMS_2014_0115_58434.pdf\page15.html NOT found
html_files\CMS_2014_0115_58434.pdf\page16.html found
html_files\CMS_2014_0115_58434.pdf\page17.html found
html_files\CMS_2014_0115_58434.pdf\page17.html NOT found
html_files\CMS_2014_0115_58434.pdf\page18.html found
html_files\CMS_2014_0115_58434.pdf\page18.html NOT found
html_files\CMS_2014_0115_58434.pdf\page19.html found
html_files\CMS_2014_0115_58434.pdf\page19.html NOT found
html_files\CMS_2014_0115_58434.pdf\page2.html found
html_files\CMS_2014_0115_58434.pdf\page2.html NOT found
html_files\CMS_2014_0115_58434.pdf\page20.html found
html_files\CMS_2014_0115_58434.pdf\page20.html NOT found
html_files\CMS_2014_0115_58434.pdf\page21.html found
html_files\CMS_2014_0115_58434.pdf\page21.html NOT found
html_files\CMS_2014_0115_58434.pdf\page3.html found
html_files\CMS_2014_0115_58434.pdf\page4.html found
html_files\CMS_2014_0115_58434.pdf\page4.html NOT found
html_files\CMS_2014

In [8]:
a = pd.concat(features, axis=0, sort=False).reset_index(drop=1)
a["num_words"] = a["text"].apply(lambda x: len(x.split()))
a["num_letters"] = a["text"].apply(lambda x: len(x))
a["first_word"] = a["text"].apply(lambda x: x.split()[0]) ## extract the first word from the text
a["last_word"] = a["text"].apply(lambda x: x.split()[-1]) ## extract the last word from the text

In [11]:
a

,class,style,id,font-size,vertical-align,color,font-family,font-weight,font-style,position,left,top,text,page,document,line-height,num_words,num_letters,first_word,last_word
0,txt,font-size:11px;vertical-align:baseline;color:r...,f2,11px,baseline,"rgba(0,0,0,1)",sans-serif,normal,normal,absolute,72px,74px,direct contraceptive access without creating a...,page6.html,html_files\CMS_2014_0115_0059.pdf,NaN,11,89,direct,"payors,"
1,txt,font-size:11px;vertical-align:baseline;color:r...,f2,11px,baseline,"rgba(0,0,0,1)",sans-serif,normal,normal,absolute,72px,88px,"administrators, agents, Departments of Insuran...",page6.html,html_files\CMS_2014_0115_0059.pdf,NaN,7,65,"administrators,",etc.
2,txt,font-size:11px;vertical-align:baseline;color:r...,f3,11px,baseline,"rgba(0,0,0,1)",sans-serif,bold,normal,absolute,72px,125px,VII. Proposed solution: Give the covered stude...,page6.html,html_files\CMS_2014_0115_0059.pdf,NaN,13,85,VII.,their
3,txt,font-size:11px;vertical-align:baseline;color:r...,f3,11px,baseline,"rgba(0,0,0,1)",sans-serif,bold,normal,absolute,72px,138px,federal income taxes for all costs incurred fo...,page6.html,html_files\CMS_2014_0115_0059.pdf,NaN,11,82,federal,methods.
4,txt,font-size:11px;vertical-align:baseline;color:r...,f2,11px,baseline,"rgba(0,0,0,1)",sans-serif,normal,normal,absolute,72px,165px,As opposed to requiring the insurance company ...,page6.html,html_files\CMS_2014_0115_0059.pdf,NaN,14,98,As,specific
5,txt,font-size:11px;vertical-align:baseline;color:r...,f2,11px,baseline,"rgba(0,0,0,1)",sans-serif,normal,normal,absolute,72px,179px,"methods of contraception, of which the eligibl...",page6.html,html_files\CMS_2014_0115_0059.pdf,NaN,15,98,methods,and
6,txt,font-size:11px;vertical-align:baseline;color:r...,f2,11px,baseline,"rgba(0,0,0,1)",sans-serif,normal,normal,absolute,72px,192px,would potentially breach the privacy of the co...,page6.html,html_files\CMS_2014_0115_0059.pdf,NaN,14,92,would,the
7,txt,font-size:11px;vertical-align:baseline;color:r...,f2,11px,baseline,"rgba(0,0,0,1)",sans-serif,normal,normal,absolute,72px,206px,covered persons to submit any expenditure for ...,page6.html,html_files\CMS_2014_0115_0059.pdf,NaN,15,99,covered,with
8,txt,font-size:11px;vertical-align:baseline;color:r...,f2,11px,baseline,"rgba(0,0,0,1)",sans-serif,normal,normal,absolute,72px,219px,their federal tax return for a full refund. Th...,page6.html,html_files\CMS_2014_0115_0059.pdf,NaN,17,102,their,craft
9,txt,font-size:11px;vertical-align:baseline;color:r...,f2,11px,baseline,"rgba(0,0,0,1)",sans-serif,normal,normal,absolute,72px,233px,and administer a whole new policy as well as c...,page6.html,html_files\CMS_2014_0115_0059.pdf,NaN,16,91,and,any
